<a href="https://colab.research.google.com/github/WesleySantosMaxtelll/doutorado/blob/main/pre_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 47.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.2 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import collections
import matplotlib.pyplot as plt
import sys
import ast
import pandas as pd
import torch
import os
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
from torch.optim import AdamW, SGD
from transformers import BertTokenizer, BertModel
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from sklearn.utils import class_weight
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import _pickle as pickle
import time
from datetime import datetime, timedelta
import gc
from torch.nn import CrossEntropyLoss

In [4]:
torch.cuda.empty_cache()
device = torch.device('cuda')
print(device)
# from keras.preprocessing.sequence import pad_sequences
source_folder = 'Data'
destination_folder = 'Model'


# Define the function for creating a weight dictionary.
def create_weight_dict(labels):
    unique_labels = np.unique(labels)
    class_weights = class_weight.compute_class_weight('balanced', classes=unique_labels, y=labels)
    return class_weights


def get_loss(outputs, b_labels, vect_weights):
    loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32))
    loss_fct.to(device)
    loss = loss_fct(outputs[1].view(-1, 2), b_labels.view(-1))
    return loss

cuda


In [5]:
class Bert(nn.Module):
    def __init__(self, device, pre_trained='neuralmind/bert-base-portuguese-cased'):
        super(Bert, self).__init__()
        self.device = device
        self.bert = BertModel.from_pretrained(pre_trained)
        self.hidden_size = self.bert.config.hidden_size
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.drop_out = nn.Dropout(0.25)
        self.fc1 = nn.Linear(100, 50)
        self.att1 = nn.Linear(self.hidden_size, 4)
        self.att2 = nn.Linear(30, 4)
        self.bn1 = nn.BatchNorm1d(50)
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100, 2)

    def forward(self, inputs):
        """
        args:
        x:  a list of inputs of diemension [BxTx6]
        """
        outputs = []
        # iterate in the batch through all sequences
        for ins in zip(inputs):
            # with torch.no_grad():
            tokens_tensor = ins[0]['tokens']
            segments_tensors = ins[0]['mask']
            tokens_tensor_split = torch.split(tokens_tensor, 500)
            segments_tensors_split = torch.split(segments_tensors, 500)
            encoded_user = []
            for t, s in zip(tokens_tensor_split, segments_tensors_split):
                encoded = self.bert(t.to(device), s.to(device))['last_hidden_state']
                encoded_user.append(encoded.cpu())
                torch.cuda.empty_cache()
            xx = torch.concat(encoded_user, dim=0)
                # xx = self.bert(ins[0]['tokens'].to(device), ins[0]['mask'].to(device))['last_hidden_state']
            xx = xx.to(device)
            dts = torch.tensor(ins[0]['diff'])
            tms = torch.tensor(ins[0]['time_vect'])
            n_posts, size_post, dim_emb = xx.shape
            decay = np.exp(-dts / dts.mean())
            weight_decay_tf = torch.multiply(tms, decay.view(-1, 1))
            # what a line
            # shift to get the previous message
            att_xx = xx.roll(1, 0)
            att_xx[0] = 0


            att_matrix1 = torch.bmm(weight_decay_tf.view(-1, 1, 4).to(device), self.att1(att_xx).view(n_posts, 4, 30)).view(-1, 30)
            att_matrix2 = torch.bmm(weight_decay_tf.view(-1, 1, 4).to(device), self.att2(att_xx.view(-1,
                                self.hidden_size, 30)).view(n_posts, 4, self.hidden_size)).view(-1, self.hidden_size)
            att = torch.multiply(att_matrix1.view(-1, 30, 1), att_matrix2.view(-1, 1, self.hidden_size))
            xx = torch.mul(att, xx)
            torch.cuda.empty_cache()
            out, hiden = self.rnn.cpu()(xx.cpu())#self.rnn(xx)
            out = out[-1, :, :].view(-1).to(device)
            out = self.fc_out3(out)
            out = F.dropout(out, 0.2)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.2)
            out = self.fc_out1(out)
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs


class BertVectorizer(nn.Module):
    def __init__(self, device, pre_trained='neuralmind/bert-base-portuguese-cased'):
        super(BertVectorizer, self).__init__()
        self.device = device
        self.bert = BertModel.from_pretrained(pre_trained)

    def forward(self, inputs):
        """
        args:
        x:  a list of inputs of diemension [BxTx6]
        """
        with torch.no_grad():
            xx = self.bert(inputs['tokens'], inputs['mask'])['last_hidden_state']
        return xx


In [6]:
class BertUser(nn.Module):
    def __init__(self, device, pre_trained='neuralmind/bert-base-portuguese-cased'):
        super(BertUser, self).__init__()
        self.device = device
        self.bert = BertModel.from_pretrained(pre_trained)
        self.hidden_size = self.bert.config.hidden_size
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.sample_size = 10
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100 * self.sample_size, 2)

    def forward(self, tokens, masks):
        outputs = []
        # iterate in the batch through all sequences
        for token, mask in zip(tokens, masks):
            encoded = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            out, hiden = self.rnn(encoded)
            out = self.fc_out3(out.view(self.sample_size, -1))
            out = F.dropout(out, 0.1)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.1)
            out = self.fc_out1(out.view(-1))
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs


In [7]:
def get_tokens_mask_dates_split(X=None, y=None, tokenizer=None, type_data=None, task=None, tl_position=None, ml=30):
    path = f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/{type_data}_{task}_{tl_position}_processed.pkl'

    if os.path.isfile(path):
        print('load', path)
        X_processed, y = pickle.load(open(path, 'rb'))
    else:
        print('Prepare')
        X_processed = []
        for user_posts in X:
            # print(tokenizer)
            tokenized_texts = tokenizer.batch_encode_plus(user_posts, max_length=ml, padding='max_length', truncation=True)
            tokens_tensor = torch.tensor(tokenized_texts['input_ids'])
            segments_tensors = torch.tensor(tokenized_texts['attention_mask'])
            X_processed.append(
                {'tokens': tokens_tensor,
                 'mask': segments_tensors,
                 }
            )
        pickle.dump([X_processed, y], open(path, 'wb'))
    return X_processed, y
  

In [8]:
path = '/content/drive/MyDrive/doutorado/dados_v6/pre_pos/'

In [9]:
def load_data(task, sub_set, tl_position):
    df_c = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/train_test/{task}/{task}_c_{tl_position}_{sub_set}.csv', sep=';')
    df_d = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/train_test/{task}/{task}_{tl_position}_{sub_set}.csv', sep=';')
    return pd.concat((df_c, df_d))

In [10]:
def load_convert(task, tl_position):
    df_test = load_data(task, 'test', tl_position)
    df_train = load_data(task, 'train', tl_position)
    df_test['Text'] = df_test['Text'].apply(lambda x: x.split('#'))
    df_test['label'] = df_test['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    df_train['Text'] = df_train['Text'].apply(lambda x: x.split('#'))
    df_train['label'] = df_train['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    return df_train, df_test

In [11]:
import random

class CustomTextDataset(Dataset):
    def __init__(self, tokens_mask, labels):
        self.tokens_mask = tokens_mask
        self.labels = labels
        self.size = 10

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        data = self.tokens_mask[idx]
        start = random.randint(0, data['tokens'].shape[0] - self.size)
        data_short = {v: data[v][start: start + self.size] for v in data}
        return [data_short, label]


In [12]:
# batch = tuple(t.to(device) for t in a)
# Unpack the inputs from our dataloader
# b_input_ids_b_input_mask, b_labels = batch
# len_input = [Counter(value.tolist())[1] for value in b_input_mask]
# outputs = model([b_input_ids, b_input_mask, len_input])


In [13]:
def save_checkpoint(save_path, model, valid_loss):
    if save_path == None:
        return

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

In [14]:
def load_checkpoint(load_path, model):
    if load_path == None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [15]:
def run(task, tl_position):

    df_train, df_test = load_convert(task, tl_position)    
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
    X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task)
    X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task)

In [16]:
os.listdir(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/')

['model-full-D-pre-epoch0.pt',
 'model-full-D-pre-epoch1.pt',
 'model-full-D-pre-epoch2.pt',
 'model-full-D-pre-epoch3.pt',
 'model-full-D-pre-epoch4.pt',
 'model-full-D-pre-epoch5.pt',
 'model-full-D-pre-epoch6.pt',
 'model-full-D-pre-epoch7.pt',
 'model-full-D-pre-epoch8.pt',
 'model-full-D-pre-epoch9.pt',
 'model-full-D-pre-epoch10.pt',
 'model-full-D-pre-epoch11.pt',
 'model-full-D-pre-epoch12.pt',
 'model-full-D-pre-epoch13.pt',
 'model-full-D-pre-epoch14.pt',
 'model-full-D-pre-epoch15.pt',
 'model-full-D-pre-epoch16.pt',
 'model-full-D-pre-epoch17.pt',
 'model-full-D-pre-epoch18.pt',
 'model-full-D-pre-epoch19.pt',
 'model-full-D-pos-epoch0.pt',
 'model-full-D-pos-epoch1.pt',
 'model-full-D-pos-epoch2.pt',
 'model-full-D-pos-epoch3.pt',
 'model-full-D-pos-epoch4.pt',
 'model-full-D-pos-epoch5.pt',
 'model-full-D-pos-epoch6.pt',
 'model-full-D-pos-epoch7.pt',
 'model-full-D-pos-epoch8.pt',
 'model-full-D-pos-epoch9.pt',
 'model-full-D-pos-epoch10.pt',
 'model-full-D-pos-epoch11.p

In [ ]:
task = 'D'
tl_position = 'pos'

df_train, df_test = load_convert(task, tl_position)    
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, tl_position)
X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, tl_position)

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

load /content/drive/MyDrive/doutorado/dados_v6/pre_pos/train_D_pos_processed.pkl
load /content/drive/MyDrive/doutorado/dados_v6/pre_pos/test_D_pos_processed.pkl


KeyboardInterrupt: ignored

In [29]:
task = 'A'
tl_position = 'pos'

df_train, df_test = load_convert(task, tl_position) 

In [19]:
from collections import Counter

In [32]:
Counter(df_test['Diagnosed_YN'])

Counter({'no': 2177, 'yes': 311})

In [ ]:
len(df_train['Text'].iloc[20])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
batch_size = 16

TD_train = CustomTextDataset(X_train, y_train)
dataloader_train = DataLoader(TD_train, batch_size=batch_size, shuffle=True, collate_fn=(lambda x: x))

TD_val = CustomTextDataset(X_val, y_val)
dataloader_val = DataLoader(TD_val, batch_size=batch_size, shuffle=True, collate_fn=(lambda x: x))


TD_test = CustomTextDataset(X_test, y_test)
dataloader_test = DataLoader(TD_test, batch_size=batch_size, shuffle=False, collate_fn=(lambda x: x))

In [ ]:
len(X_val)

In [ ]:
device = 'cuda'
## Define model
model = BertUser(device)
model.to(device)
lr = 2e-5
### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr)  # To reproduce BertAdam specific behavior set correct_bias=False
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler


In [ ]:

# Store our loss and accuracy for plotting
train_loss_set = []

epochs = 0
file_path = '/content/drive/MyDrive/doutorado/dados_v6/pre_pos/model'
vect_weights = create_weight_dict(y_train)
loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32), reduction='mean')
loss_fct.to(device)
for epoch in range(epochs):
    print(f'Epoch {epoch}')
    model.train()
    pred, true_label = [], []
    print(len(dataloader_train))
    for i, batch in enumerate(dataloader_train):
        if i % 100 == 0:
            print(datetime.fromtimestamp(time.time()), i, train_loss_set[-1] if len(train_loss_set) else 0)
        atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
        masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
        outputs = model(atts, masks)
        loss = loss_fct(outputs, torch.tensor([b[1] for b in batch]).to(device))
        # output = loss(torch.argmax(outputs[1], dim=1).add_(-1), b_labels.add_(-1))
        train_loss_set.append(loss.item())
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # scheduler.step()
        optimizer.zero_grad()
        pred = pred + outputs.cpu().argmax(1).tolist()
        true_label = true_label + [b[1] for b in batch]
        gc.collect()
        torch.cuda.empty_cache()
    print('Results')
    print(classification_report(true_label, pred))
    # continue
    model.eval()
    with torch.no_grad():
        # validation loop
        pred, real = [], []
        for j, batch in enumerate(dataloader_val):
            atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
            masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
            outputs = model(atts, masks)

            pred = pred + outputs.cpu().argmax(1).tolist()
            real = real + [b[1] for b in batch]
        print(classification_report(real, pred))
        save_checkpoint(file_path + '/' + f'model-full-{task}-{tl_position}-epoch{epoch}.pt', model, 0)
        # save_metrics(file_path + '/' + f'metrics-full-{task}-epoch{epoch}.pt', 0,0,0)
    model.train()


In [ ]:
all_models = os.listdir(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/')

In [ ]:
all_models

['model-full-D-pre-epoch0.pt',
 'model-full-D-pre-epoch1.pt',
 'model-full-D-pre-epoch2.pt',
 'model-full-D-pre-epoch3.pt',
 'model-full-D-pre-epoch4.pt',
 'model-full-D-pre-epoch5.pt',
 'model-full-D-pre-epoch6.pt',
 'model-full-D-pre-epoch7.pt',
 'model-full-D-pre-epoch8.pt',
 'model-full-D-pre-epoch9.pt',
 'model-full-D-pre-epoch10.pt',
 'model-full-D-pre-epoch11.pt',
 'model-full-D-pre-epoch12.pt',
 'model-full-D-pre-epoch13.pt',
 'model-full-D-pre-epoch14.pt',
 'model-full-D-pre-epoch15.pt',
 'model-full-D-pre-epoch16.pt',
 'model-full-D-pre-epoch17.pt',
 'model-full-D-pre-epoch18.pt',
 'model-full-D-pre-epoch19.pt',
 'model-full-D-pos-epoch0.pt',
 'model-full-D-pos-epoch1.pt',
 'model-full-D-pos-epoch2.pt',
 'model-full-D-pos-epoch3.pt',
 'model-full-D-pos-epoch4.pt',
 'model-full-D-pos-epoch5.pt',
 'model-full-D-pos-epoch6.pt',
 'model-full-D-pos-epoch7.pt',
 'model-full-D-pos-epoch8.pt',
 'model-full-D-pos-epoch9.pt',
 'model-full-D-pos-epoch10.pt',
 'model-full-D-pos-epoch11.p

In [ ]:
for i in range(20):
    p = f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-{task}-{tl_position}-epoch{i}.pt'
    if os.path.isfile(p):
        load_checkpoint(p, model)
        model.eval()
        with torch.no_grad():
            # validation loop
            pred, real = [], []
            for j, batch in enumerate(dataloader_val):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))


Model loaded from <== /content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-D-pos-epoch0.pt
              precision    recall  f1-score   support

           0       0.96      0.56      0.71      1248
           1       0.19      0.80      0.31       164

    accuracy                           0.59      1412
   macro avg       0.57      0.68      0.51      1412
weighted avg       0.87      0.59      0.66      1412

Model loaded from <== /content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-D-pos-epoch1.pt
              precision    recall  f1-score   support

           0       0.95      0.68      0.79      1248
           1       0.22      0.70      0.34       164

    accuracy                           0.68      1412
   macro avg       0.58      0.69      0.56      1412
weighted avg       0.86      0.68      0.74      1412

Model loaded from <== /content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-D-pos-epoch2.pt
              precision    reca

In [ ]:
best_settings = [
  "A pre 6",
  "A pos 12",
  "D pre 12",
  "D pos 7"
]
device = 'cuda'
## Define model
model = BertUser(device)
model.to(device)

for bs in best_settings:
    print(bs)
    task, tl_position, i = bs.split()
    p = f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-{task}-{tl_position}-epoch{i}.pt'
    if os.path.isfile(p):
        load_checkpoint(p, model)
        model.eval()
        with torch.no_grad():
            # validation loop
            df_train, df_test = load_convert(task, tl_position)    
            tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
            X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, tl_position)
            X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, tl_position)
            batch_size = 16
            TD_test = CustomTextDataset(X_test, y_test)
            dataloader_test = DataLoader(TD_test, batch_size=batch_size, shuffle=False, collate_fn=(lambda x: x))
            all_pred = []
            for _ in range(50):
                pred, real = [], []
                for j, batch in enumerate(dataloader_test):
                    atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                    masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                    outputs = model(atts, masks)

                    pred = pred + outputs.cpu().argmax(1).tolist()
                    real = real + [b[1] for b in batch]
                print(classification_report(real, pred))
                all_pred.append(pred)
            p = (np.array(all_pred).mean(0) > 0.5).astype(int)
            print(classification_report(real, p))
            d = {
              'pred': p,
              'pred_all': np.array(all_pred).mean(0),
              'real': real
            }
            df = pd.DataFrame(d)
            pickle.dump(df, open(f'/content/drive/MyDrive/doutorado/dados_v6/pre_pos/prediction/{task}_{tl_position}.pkl', 'wb'))
            


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


A pre 6
Model loaded from <== /content/drive/MyDrive/doutorado/dados_v6/pre_pos/model/model-full-A-pre-epoch6.pt
load /content/drive/MyDrive/doutorado/dados_v6/pre_pos/train_A_pre_processed.pkl
load /content/drive/MyDrive/doutorado/dados_v6/pre_pos/test_A_pre_processed.pkl
              precision    recall  f1-score   support

           0       0.90      0.79      0.84      2177
           1       0.22      0.40      0.28       311

    accuracy                           0.74      2488
   macro avg       0.56      0.60      0.56      2488
weighted avg       0.82      0.74      0.77      2488

              precision    recall  f1-score   support

           0       0.90      0.80      0.84      2177
           1       0.21      0.38      0.27       311

    accuracy                           0.74      2488
   macro avg       0.56      0.59      0.56      2488
weighted avg       0.81      0.74      0.77      2488

              precision    recall  f1-score   support

           0     

In [ ]:
p = (np.array(all_pred).mean(0) > 0.4).astype(int)

In [ ]:
print(classification_report(real, p))


              precision    recall  f1-score   support

           0       0.92      0.91      0.91      2177
           1       0.41      0.45      0.43       311

    accuracy                           0.85      2488
   macro avg       0.66      0.68      0.67      2488
weighted avg       0.86      0.85      0.85      2488



In [ ]:
a[0][0]['tokens'], a[0][0]['mask']

In [ ]:
pre_trained='neuralmind/bert-base-portuguese-cased'
bert = BertModel.from_pretrained(pre_trained)

In [ ]:
m = BertUser('')

In [ ]:
m(atts, masks)

In [ ]:
out = bert(input_ids=atts[0], attention_mask=masks[0])

In [ ]:
rnn = nn.LSTM(input_size=768, hidden_size=100,
                           num_layers=4, bidirectional=True)

In [ ]:
out_rnn = rnn(out['last_hidden_state'])

In [ ]:
out_rnn[0].view(10, -1).shape

In [ ]:
fc_out3 = nn.Linear(6000, 1000)
fc_out2 = nn.Linear(1000, 100)
fc_out1 = nn.Linear(1000, 2)

In [ ]:
fc_out1(fc_out2(fc_out3(out_rnn[0].view(10, -1))).view(-1))

In [ ]:
atts = torch.stack([aa[0]['tokens'] for aa in a])
masks = torch.stack([aa[0]['mask'] for aa in a])

In [ ]:
masks.shape

In [ ]:
out['last_hidden_state']

In [ ]:
X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, tl_position)

In [ ]:
len(X_test)

In [ ]:
# tokenizer(df_test)
t = get_tokens_mask_dates_split(df_test['Text'].iloc[:10], np.array([1]), tokenizer=tokenizer)

In [ ]:
t[1]